## Introduction to Rates

Start every session with authenticating with your unique client id and secret. If you don't have a registered app, create one [here](https://marquee.gs.com/s/developer/myapps/register). `run_analytics` scope is required for the functionality covered in this example.

In [1]:
from gs_quant.session import GsSession, Environment
GsSession.use(Environment.PROD, client_id=None, client_secret=None, scopes=('run_analytics',)) 

Let’s now create IR Swaptions for a number of strikes, expiries and tenors. This will create 64 swaptions (4 strikes x 4 tenors x 4 expiries). All the available instruments can be [viewed here](https://developer.gs.com/docs/gsquant/guides/Pricing-and-Risk/instruments/#supported-instruments).

In [2]:
from gs_quant.common import Currency, PayReceive
from gs_quant.instrument import IRSwaption

strikes = ['ATMF-10', 'ATMF+10', 'ATMF+50','ATMF+100']
tenors = ['1y', '2y', '5y', '10y']
expiries = ['3m', '6m', '1y', '3y']

swaptions = [ IRSwaption(PayReceive.Receive, tenor, Currency.USD, expirationDate=expiry, strike=strike, notionalAmount=1e8) 
             for strike in strikes for tenor in tenors for expiry in expiries ]

len(swaptions)

64

We can now calculate a number of risk measures for these swaptions. Let’s calculate dollar price and vega in a single call using pricing context and retrieve the result. All the available measures can be [viewed here](https://developer.gs.com/docs/gsquant/guides/Pricing-and-Risk/measures/).

In [3]:
from gs_quant.markets import PricingContext
from gs_quant.risk import IRVega

with PricingContext():
    price_f = [swaption.dollar_price() for swaption in swaptions]
    vega_f = [swaption.calc(IRVega) for swaption in swaptions]

prices = [f.result() for f in price_f]
vegas = [f.result() for f in vega_f]

Let’s now construct a grid summarizing our results and sort risk.

In [4]:
import pandas as pd
vegas_frame = pd.concat(vegas).reset_index(drop=True).rename(columns={'value': 'vega'})
swaption_params = [ [ x.expirationDate, x.terminationDate, x.strike, prices[i] ] for i, x in enumerate(swaptions) ]
swaption_frame = pd.DataFrame(swaption_params, columns=['expiry', 'tenor', 'strike', 'price'])

In [5]:
from gs_quant.risk import sort_risk

result = pd.concat([vegas_frame, swaption_frame],axis=1)
sort_risk(result).head(10) 

,marketDataType,assetId,pointClass,point,vega,expiry,tenor,strike,price
0,IR VOL,USD-LIBOR-BBA,SWAPTION,1Y;3M,1637.735726,3m,1y,ATMF-10,29279.861182
1,IR VOL,USD-LIBOR-BBA,SWAPTION,1Y;3M,1575.957990,3m,1y,ATMF+10,122832.344562
2,IR VOL,USD-LIBOR-BBA,SWAPTION,1Y;3M,26.713023,3m,1y,ATMF+50,492588.795907
3,IR VOL,USD-LIBOR-BBA,SWAPTION,1Y;3M,0.114935,3m,1y,ATMF+100,984955.333067
4,IR VOL,USD-LIBOR-BBA,SWAPTION,1Y;6M,2605.643265,6m,1y,ATMF-10,73542.011651
5,IR VOL,USD-LIBOR-BBA,SWAPTION,1Y;6M,2590.462164,6m,1y,ATMF+10,166013.487734
6,IR VOL,USD-LIBOR-BBA,SWAPTION,1Y;6M,528.261783,6m,1y,ATMF+50,495247.605509
7,IR VOL,USD-LIBOR-BBA,SWAPTION,1Y;6M,23.317284,6m,1y,ATMF+100,981609.184716
8,IR VOL,USD-LIBOR-BBA,SWAPTION,1Y;1Y,3815.760705,1y,1y,ATMF-10,161377.202961
9,IR VOL,USD-LIBOR-BBA,SWAPTION,1Y;1Y,3814.458256,1y,1y,ATMF+10,253025.741128


Please reach out to `gs-quant@gs.com` with any questions.

### Disclaimer
This website may contain links to websites and the content of third parties ("Third Party Content"). We do not monitor, review or update, and do not have any control over, any Third Party Content or third party websites. We make no representation, warranty or guarantee as to the accuracy, completeness, timeliness or reliability of any Third Party Content and are not responsible for any loss or damage of any sort resulting from the use of, or for any failure of, products or services provided at or from a third party resource. If you use these links and the Third Party Content, you acknowledge that you are doing so entirely at your own risk.